# Data Validation
Author: Ines LU <br>
Date: 13/01/2024 <br>
Subject: Sales Rep reporting to the executive team  <br> 

## Import Library

In [5]:
import pandas as pd

## Data Overview

In [6]:
df = pd.read_csv('raw/product_sales.csv')

In [7]:
# Dataset Overview
print("Dataset Overview:")
print(f"Number of Rows: {df.shape[0]}")
print(f"Number of Columns: {df.shape[1]}")
print()

# Checking for missing values, data types, and duplicates
missing_values = df.isnull().sum()
data_types = df.dtypes
duplicate_entries = df.duplicated().sum()

# Summary of data validation checks
validation_summary = pd.DataFrame({'Missing Values': missing_values, 'Data Types': data_types})
validation_summary['Duplicates'] = duplicate_entries if duplicate_entries > 0 else "No duplicates"
print("validation_summary:")
print(validation_summary)


Dataset Overview:
Number of Rows: 15000
Number of Columns: 8

validation_summary:
                   Missing Values Data Types     Duplicates
week                            0      int64  No duplicates
sales_method                    0     object  No duplicates
customer_id                     0     object  No duplicates
nb_sold                         0      int64  No duplicates
revenue                      1074    float64  No duplicates
years_as_customer               0      int64  No duplicates
nb_site_visits                  0      int64  No duplicates
state                           0     object  No duplicates


In [9]:
# Data Distribution (for numerical columns)
print("Data Distribution (Numerical Columns):\n")

data_distribution = pd.DataFrame()
for column in df.select_dtypes(include=['number']).columns:
    description = df[column].describe()[['count', 'mean', 'std', 'min', 'max']]
    data_distribution[column] = description
    
# Transpose the DataFrame 
data_distribution = data_distribution.T
print(data_distribution)

Data Distribution (Numerical Columns):

                     count       mean        std    min     max
week               15000.0   3.098267   1.656420   1.00    6.00
nb_sold            15000.0  10.084667   1.812213   7.00   16.00
revenue            13926.0  93.934943  47.435312  32.54  238.32
years_as_customer  15000.0   4.965933   5.044952   0.00   63.00
nb_site_visits     15000.0  24.990867   3.500914  12.00   41.00


In [10]:
# Categorical Data (for categorical columns)
print("Categorical Data (Categorical Columns):")
for column in df.select_dtypes(include=['object']).columns:
    print(f"{column}:")
    print(df[column].value_counts())
    print()

Categorical Data (Categorical Columns):
sales_method:
Email           7456
Call            4962
Email + Call    2549
em + call         23
email             10
Name: sales_method, dtype: int64

customer_id:
2e72d641-95ac-497b-bbf8-4861764a7097    1
e0c955c5-50b4-43dc-bfdd-e181ac3c8e50    1
8a7a36dd-8fdc-4d08-b87d-4f1548f723be    1
f7c7fcbe-ac44-4e62-a470-b04eb19fb5a1    1
9149e07e-cd2f-4a94-a97b-2f2f475afdbb    1
                                       ..
c1779213-7d49-494b-9aec-f488290bff5a    1
16d14cab-0900-4759-9c18-8569dae8c711    1
3cd54608-5e34-4340-976e-61725b41d175    1
ede3a4b7-94bf-46b2-8bd2-55e009b7c0a9    1
4e077235-7c17-4054-9997-7a890336a214    1
Name: customer_id, Length: 15000, dtype: int64

state:
California        1872
Texas             1187
New York           965
Florida            904
Illinois           617
Pennsylvania       598
Ohio               566
Michigan           498
Georgia            489
North Carolina     459
New Jersey         434
Virginia           372
I

In [11]:
# Calculate the count of unique values in the 'state' column
unique_state_count = df['state'].nunique()
unique_state_count

50

## Data Cleaning

In [12]:
# Drop rows where 'revenue' is null
df.dropna(subset=['revenue'], inplace=True)

In [13]:
# extreme values in years_as_customer
df_filter = df[df['years_as_customer'] > 40]
df_filter

,week,sales_method,customer_id,nb_sold,revenue,years_as_customer,nb_site_visits,state
13741,2,Email,18919515-a618-430c-9a05-2c7d8fea96af,10,97.22,63,24,California
13800,4,Call,2ea97d34-571d-4e1b-95be-fea1c404649f,10,50.47,47,27,California


In [14]:
# drop rows where ['years_as_customer'] > 40
df = df[df['years_as_customer'] <= 40]

In [15]:
# sales method aggregation
method_mapping = {
    'Email': ['Email', 'email', 'em + call'],
    'Call': ['Call'],
    'Email + Call': ['Email + Call']
}

df['combined_sales_method'] = df['sales_method'].apply(lambda x: next((key for key, values in method_mapping.items() if x.lower() in values), x))

df.drop(columns=['sales_method'], inplace=True)

In [16]:
df['combined_sales_method'].unique()

array(['Email + Call', 'Call', 'Email'], dtype=object)

In [17]:
df.head()

,week,customer_id,nb_sold,revenue,years_as_customer,nb_site_visits,state,combined_sales_method
1,6,3998a98d-70f5-44f7-942e-789bb8ad2fe7,15,225.47,1,28,Kansas,Email + Call
2,5,d1de9884-8059-4065-b10f-86eef57e4a44,11,52.55,6,26,Wisconsin,Call
4,3,10e6d446-10a5-42e5-8210-1b5438f70922,9,90.49,0,28,Illinois,Email
5,6,6489e678-40f2-4fed-a48e-d0dff9c09205,13,65.01,10,24,Mississippi,Call
6,4,eb6bd5f1-f115-4e4b-80a6-5e67fcfbfb94,11,113.38,9,28,Georgia,Email


In [18]:
df.shape

(13924, 8)

In [19]:
df.to_csv('product_sales_cleaned.csv', index=False)